In [1]:
from kafka import KafkaConsumer
import json
import matplotlib.pyplot as plt

city_count = 0
village_count = 0
male_count = 0
female_count = 0
age_count = 0
age_value = 0
age_min = 90
age_max = 0
male_age_sum = 0
female_age_sum = 0
male_age_mean = 0 
female_age_mean = 0 
value_mean = 0
value_sum = 0
value_mean_history = []

def process_transaction(transaction):
    print(transaction)
    
    global city_count, village_count, male_count, female_count, age_count, age_value, age_min, age_max, value_mean, value_sum, male_age_sum, female_age_sum, male_age_mean, female_age_mean
    
    transaction_str = transaction.decode('utf-8')
    transaction_json = json.loads(transaction_str)
    
    if transaction_json['location'] == 'city':
        city_count += 1
    elif transaction_json['location'] == 'village':
        village_count += 1
    
    if transaction_json['sex'] == 'M':
        male_count += 1
        male_age_sum += transaction_json['age']
    elif transaction_json['sex'] == 'F':
        female_count += 1
        female_age_sum += transaction_json['age']
    
    age_count += 1
    age_value += transaction_json['age']
    print(age_value)
    age_mean=age_value/age_count
    
    if transaction_json['age'] < age_min:
        age_min = transaction_json['age']
    elif transaction_json['age'] > age_max:
        age_max = transaction_json['age']
        
    value_sum += transaction_json['value']
    
    value_mean = value_sum/age_count
    value_mean_history.append(value_mean)
    
    if(male_count > 0):
        male_age_mean = male_age_sum/male_count
    if(female_count > 0):
        female_age_mean = female_age_sum/female_count
        
    print("Liczba mieszkańców miast: ",city_count)
    print("Liczba mieszkańców wsi: ",village_count)
    print("Liczba mężczyzn: ",male_count)
    print("Liczba kobiet: ",female_count)
    print("Średni wiek: ",round(age_mean, 0))
    print("Minimalny wiek: ",age_min)
    print("Maksymalny wiek: ",age_max)
    print("Łączna wartość zakupów: ",value_sum)
    print("Średnia wartość zakupów dotychczas: ",round(value_mean, 2))
    
    # Wykresy
    x = ['City', 'Village']
    city_village_counts = [city_count, village_count]
    city_village_colors = ['#a45e6e', '#eb8898']
    
    genders = ['Male', 'Female']
    gender_counts = [male_count, female_count]
    gender_colors = ['#098aac', '#defdf4']
    
    age_stats_labels = ['Mean Age', 'Male Mean Age', 'Female Mean Age']
    age_stats_values = [age_mean, male_age_mean, female_age_mean]
    age_stats_colors = ['#1e5550', '#c1e1a5', '#75a47f']
    
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.bar(x, city_village_counts, color=city_village_colors)
    plt.title('City vs. Village Counts')
    plt.xlabel('Location')
    plt.ylabel('Count')
    
    plt.subplot(2, 2, 2)
    plt.pie(gender_counts, labels=genders, colors=gender_colors, autopct='%1.1f%%')
    plt.title('Gender Distribution')
    
    plt.subplot(2, 2, 3)
    plt.bar(age_stats_labels, age_stats_values, color=age_stats_colors)
    plt.title('Age Statistics')
    plt.ylim(bottom=0, top=90)
    plt.xlabel('Statistic')
    plt.ylabel('Age')
    
    plt.subplot(2, 2, 4)
    plt.plot(value_mean_history, marker='o', color='#ff7653')  # Wykres liniowy z historią wartości
    plt.title('Mean Values of Purchases')
    plt.xlabel('Transaction')
    plt.ylabel('Mean Value')
    
    plt.tight_layout()
    plt.show()

def consume_transactions():
    consumer = KafkaConsumer('transactions-topic', bootstrap_servers='broker:9092')
    for message in consumer:
        transaction = message.value
        process_transaction(transaction)

if __name__ == '__main__':
    consume_transactions()


NoBrokersAvailable: NoBrokersAvailable